In [4]:
# package
import sys
import os
import subprocess

from six import string_types

# Make sure you have all of these packages installed, e.g. via pip
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display
import cv2
%matplotlib inline

In [3]:
PLANET_KAGGLE_ROOT = os.path.abspath("./")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train_v2.csv')
assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)

In [5]:
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
labels_df.head()
label_list = ['agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down', 'clear',
              'cloudy', 'conventional_mine', 'cultivation', 'habitation', 'haze', 'partly_cloudy',
              'primary', 'road', 'selective_logging', 'slash_burn', 'water']

In [6]:
# Add onehot features for every label
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
# Display head
labels_df.head()

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [7]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down']

In [69]:
weather_labels_value = np.transpose(np.float32(labels_df[weather_labels].values))

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [1]:
#train weather label
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import re
import random
import math

import vgg16_trainable as vgg16
import read_data
import utils
import csv

from sklearn.metrics import fbeta_score

def f2_score(y_true, y_pred):
    # fbeta_score throws a confusing error if inputs are not numpy arrays
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # We need to use average='samples' here, any other average method will generate bogus results
    return fbeta_score(y_true, y_pred, beta=2, average='samples')

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

class Config():
    batch_size = 8
    steps = "-1"
    gpu = '/gpu:0'

    # checkpoint path and filename
    logdir = "./log"
    params_dir = "./params/"
    load_filename = "vgg16.npy"
    save_filename = params_dir + "vgg16.npy"

    # path
    imgs_path = "./train-jpg/"
    labels_file = "./train_v2_bin.csv"

    # iterations config
    max_iteration = 5000
    summary_iters = 50
    # refer to synset.txt for the order of labels
    # 6: clear, 7: cloudy, 11: haze, 12:partly_cloudy
    usecols = [6, 7, 11, 12]
config = Config()
reader = read_data.Reader(config)
images = tf.placeholder(tf.float32, [config.batch_size, 224, 224, 3])
true_out = tf.placeholder(tf.float32, [config.batch_size, len(config.usecols)])
train_mode = tf.placeholder(tf.bool)

vgg = vgg16.Vgg16(config.load_filename, output_size=4)
vgg.build(images, train_mode)
print vgg.get_var_count() , "variables"
with tf.name_scope('loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(true_out * tf.log(vgg.prob), [1]))
    tf.summary.scalar('loss', cost)
    
with tf.name_scope('train'):
    rate = 1e-3
    train = tf.train.GradientDescentOptimizer(rate).minimize(cost)
    tf.summary.scalar('learning_rate', rate)
    tf.summary.scalar('batch_size', config.batch_size)
    
    merged = tf.summary.merge_all()

with tf.device(config.gpu):    
    sess = tf.Session()
    writer = tf.summary.FileWriter(config.logdir, sess.graph)
    sess.run(tf.global_variables_initializer())

    print "start training"
    # start training
    for idx in xrange(config.max_iteration):
        imgs, labels = reader.random_batch()
        # feed data into the model
        feed_dict = {
            images : imgs,
            true_out : labels,
            train_mode : True
        }
        sess.run(train, feed_dict=feed_dict)
        if  idx % 50 == 0:
            result = sess.run(merged, feed_dict=feed_dict)
            loss = sess.run(cost, feed_dict=feed_dict)
            
            # f2 score
            prob = sess.run(vgg.prob, feed_dict=feed_dict)
            y_pred = []
            for p in prob:
                temp = np.zeros(len(p))
                temp[np.argmax(p)] = 1
                y_pred.append(temp)
            y_pred = np.array(y_pred)
            f2 = f2_score(sess.run(true_out, feed_dict=feed_dict), y_pred)
            
            print idx, "cost:", loss, "f2_score:", f2
            writer.add_summary(result, idx)
            if idx % 500 == 0:
                vgg.save_npy(sess, config.save_filename)


conv5_1 (3, 3, 512, 512) (512,)
fc6 (25088, 4096) (4096,)
conv5_3 (3, 3, 512, 512) (512,)
fc7 (4096, 4096) (4096,)
fc8 (4096, 1000) (1000,)
conv5_2 (3, 3, 512, 512) (512,)
conv4_1 (3, 3, 256, 512) (512,)
conv4_2 (3, 3, 512, 512) (512,)
conv4_3 (3, 3, 512, 512) (512,)
conv3_3 (3, 3, 256, 256) (256,)
conv3_2 (3, 3, 256, 256) (256,)
conv3_1 (3, 3, 128, 256) (256,)
conv1_1 (3, 3, 3, 64) (64,)
conv1_2 (3, 3, 64, 64) (64,)
conv2_2 (3, 3, 128, 128) (128,)
conv2_1 (3, 3, 64, 128) (128,)
138361548 variables
start training
0 cost: 0.827964 f2_score: 0.75
('file saved', './params/vgg16.npy')
50 cost: 0.449362 f2_score: 0.875
100 cost: 0.0558221 f2_score: 1.0
150 cost: 0.104273 f2_score: 1.0
200 cost: 0.0804461 f2_score: 1.0
250 cost: 0.230604 f2_score: 0.875
300 cost: 0.0874491 f2_score: 1.0
350 cost: 0.0915714 f2_score: 1.0
400 cost: 0.118708 f2_score: 1.0
450 cost: 0.0930669 f2_score: 1.0
500 cost: 0.0744401 f2_score: 1.0
('file saved', './params/vgg16.npy')
550 cost: 0.0783192 f2_score: 1.0
60

In [24]:
testy = [[1,2,3],[1,2,3]]
predy = []
for p in testy:
    temp = np.zeros(len(p))
    temp[np.argmax(p)] = 1
    predy.append(temp)
print np.array(predy)

[[ 0.  0.  1.]
 [ 0.  0.  1.]]
